In [257]:
import requests
import json

# Step 1: Perform an HTTP GET request
get_url = 'https://candidate.hubteam.com/candidateTest/v3/problem/test-dataset?userKey=d173419e5afe26691b3a8a08157a'
response = requests.get(get_url)

# Ensure the request was successful
if response.status_code == 200:
    # Step 2: Process the JSON data
    data = response.json()
    # Example: Extract a specific value, e.g., 'key_to_extract'
    processed_data = data.get('key_to_extract')
else:
    print("Failed to retrieve data:", response.status_code)
    exit()

In [266]:
requests.get('https://candidate.hubteam.com/candidateTest/v3/problem/test-dataset-answer?userKey=d173419e5afe26691b3a8a08157a').json()

{'validAssociations': [{'companyId': 456, 'contactId': 1, 'role': 'EMPLOYEE'}],
 'invalidAssociations': [{'companyId': 123,
   'contactId': 1,
   'role': 'INVESTOR',
   'failureReason': 'WOULD_EXCEED_LIMIT'},
  {'companyId': 123,
   'contactId': 1,
   'role': 'FOUNDER',
   'failureReason': 'WOULD_EXCEED_LIMIT'},
  {'companyId': 123,
   'contactId': 4,
   'role': 'EMPLOYEE',
   'failureReason': 'ALREADY_EXISTS'},
  {'companyId': 123,
   'contactId': 5,
   'role': 'EMPLOYEE',
   'failureReason': 'WOULD_EXCEED_LIMIT'},
  {'companyId': 123,
   'contactId': 6,
   'role': 'EMPLOYEE',
   'failureReason': 'WOULD_EXCEED_LIMIT'}]}

In [258]:
data

{'existingAssociations': [{'companyId': 123,
   'contactId': 1,
   'role': 'EMPLOYEE'},
  {'companyId': 123, 'contactId': 2, 'role': 'EMPLOYEE'},
  {'companyId': 123, 'contactId': 3, 'role': 'EMPLOYEE'},
  {'companyId': 123, 'contactId': 4, 'role': 'EMPLOYEE'}],
 'newAssociations': [{'companyId': 123, 'contactId': 4, 'role': 'EMPLOYEE'},
  {'companyId': 123, 'contactId': 5, 'role': 'EMPLOYEE'},
  {'companyId': 123, 'contactId': 6, 'role': 'EMPLOYEE'},
  {'companyId': 123, 'contactId': 1, 'role': 'INVESTOR'},
  {'companyId': 123, 'contactId': 1, 'role': 'FOUNDER'},
  {'companyId': 456, 'contactId': 1, 'role': 'EMPLOYEE'}]}

In [259]:
import pandas as pd
temp_df = pd.DataFrame.from_dict(data, orient='index')
existing_df = pd.DataFrame(temp_df.iloc[0])
# Convert the dictionary column into separate columns
existing_df1 = pd.json_normalize(existing_df['existingAssociations'])
existing_df2 = existing_df1.dropna()
existing_df2['companyId'] = existing_df2['companyId'].apply(lambda x: str(x).rsplit('.')[0])
existing_df2['contactId'] = existing_df2['contactId'].apply(lambda x: str(x).rsplit('.')[0])

C:\Users\piush\AppData\Local\Temp\ipykernel_19328\780510407.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  existing_df2['companyId'] = existing_df2['companyId'].apply(lambda x: str(x).rsplit('.')[0])
C:\Users\piush\AppData\Local\Temp\ipykernel_19328\780510407.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  existing_df2['contactId'] = existing_df2['contactId'].apply(lambda x: str(x).rsplit('.')[0])


In [260]:
new_df = pd.DataFrame(temp_df.iloc[1])
# Convert the dictionary column into separate columns
new_df1 = pd.json_normalize(new_df['newAssociations'])
new_df1['companyId'] = new_df1['companyId'].apply(lambda x: str(x).rsplit('.')[0])
new_df1['contactId'] = new_df1['contactId'].apply(lambda x: str(x).rsplit('.')[0])

In [261]:
combined_df = pd.concat([existing_df2, new_df1], ignore_index=True)

In [262]:
def check_condition_1(df):
    """Check if at most five contacts are associated with a given role at a company."""
    grouped = df.groupby(['companyId', 'role'])
    
    def check_group(group):
        if len(group) >= 5:
            group['failureReason'] = 'WOULD_EXCEED_LIMIT'
        else:
            group['failureReason'] = ''
            #group.loc[group.index[:5], 'failureReason'] = ''
        return group

    return grouped.apply(check_group).reset_index(drop=True)

def check_condition_2(df):
    """Check if at most two roles are associated with a given contact at a company."""
    grouped = df.groupby(['companyId', 'contactId'])
    
    def check_group(group):
        if len(group) >= 2:
            group['failureReason'] = 'WOULD_EXCEED_LIMIT'
        else:
            group['failureReason'] = ''
            #group.loc[group.index[:2], 'failureReason'] = ''
        return group

    return grouped.apply(check_group).reset_index(drop=True)

# Check conditions
result_1 = check_condition_1(combined_df)
result_2 = check_condition_2(combined_df)



C:\Users\piush\AppData\Local\Temp\ipykernel_19328\1686283215.py:13: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return grouped.apply(check_group).reset_index(drop=True)
C:\Users\piush\AppData\Local\Temp\ipykernel_19328\1686283215.py:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  return grou

In [263]:
# Combine results
final_result = pd.merge(result_1, result_2, on=['companyId', 'contactId', 'role'], suffixes=('_1', '_2'))
final_result['failureReason'] = final_result.apply(lambda row: row['failureReason_1'] or row['failureReason_2'], axis=1)
final_result = final_result.drop(['failureReason_1', 'failureReason_2'], axis=1)



In [245]:


# Check for duplicates and mark as ALREADY_EXISTS
duplicates = final_result.duplicated(subset=['companyId', 'contactId', 'role'], keep='first')
final_result.loc[duplicates, 'failureReason'] = 'ALREADY_EXISTS'



In [246]:
final_result1 = final_result.merge(new_df1).drop_duplicates(subset = ['role','contactId'] , keep = 'last')

In [247]:


# Separate valid and invalid associations
valid_associations = final_result1[final_result1['failureReason'] == ''].drop('failureReason', axis=1)

invalid_associations = final_result1[final_result1['failureReason'] != '']

In [248]:
invalid_associations

,companyId,contactId,role,failureReason
3,20799,20768,INVESTOR,ALREADY_EXISTS
7,13182,13131,INVESTOR,ALREADY_EXISTS
11,22442,22464,ADVISOR,ALREADY_EXISTS
15,1390,1380,FOUNDER,ALREADY_EXISTS
19,2317,2346,BOARD_MEMBER,ALREADY_EXISTS
...,...,...,...,...
492,25440,25476,ALUMNI,WOULD_EXCEED_LIMIT
493,3153,3154,ADVISOR,WOULD_EXCEED_LIMIT
494,15226,15267,BOARD_MEMBER,WOULD_EXCEED_LIMIT
497,22743,22710,ADVISOR,WOULD_EXCEED_LIMIT


In [249]:
# Open the file in write mode
with open('valid_output.json', 'w') as json_file:
    # Write the data to the file
    json.dump({'validAssociations': valid_associations.to_json(orient='records')}, json_file, indent=4)  

In [250]:
with open('invalid_output.json', 'w') as json_file:
    # Write the data to the file
    json.dump({'invalidAssociations': invalid_associations.to_json(orient='records')}, json_file, indent=4)  

In [251]:
import json

# Load the first JSON file
with open('valid_output.json', 'r') as f:
    data1 = json.load(f)

# Load the second JSON file
with open('invalid_output.json', 'r') as f:
    data2 = json.load(f)

# Append the contents of data2 to data1
if isinstance(data1, list):
    data1.extend(data2)  # If both are lists, append the second list to the first
else:
    # If both are objects, you can't directly append one to the other
    # This example assumes you want to merge them as shown in the first example
    merged_data = {**data1, **data2}

# Write the merged data back to the first file
with open('file1.json', 'w') as f:
    json.dump(merged_data, f, indent=4)

In [252]:
# Load the second JSON file
with open('file1.json', 'r') as f:
    post_data = json.load(f)

In [253]:
post_data

{'validAssociations': '[{"companyId":"6145","contactId":"6116","role":"EMPLOYEE"},{"companyId":"25269","contactId":"25232","role":"EMPLOYEE"},{"companyId":"19716","contactId":"19765","role":"FOUNDER"},{"companyId":"8479","contactId":"8485","role":"ALUMNI"},{"companyId":"21026","contactId":"21084","role":"ADVISOR"},{"companyId":"19202","contactId":"19267","role":"ADVISOR"},{"companyId":"12333","contactId":"12354","role":"EMPLOYEE"},{"companyId":"8332","contactId":"8308","role":"INVESTOR"},{"companyId":"18679","contactId":"18685","role":"FOUNDER"},{"companyId":"2723","contactId":"2701","role":"ADVISOR"},{"companyId":"9783","contactId":"9711","role":"INVESTOR"},{"companyId":"5241","contactId":"5216","role":"EMPLOYEE"},{"companyId":"16247","contactId":"16268","role":"ADVISOR"},{"companyId":"10056","contactId":"10025","role":"ALUMNI"},{"companyId":"8022","contactId":"8099","role":"INVESTOR"},{"companyId":"16180","contactId":"16131","role":"ALUMNI"},{"companyId":"19882","contactId":"19815","

In [254]:
import json


# Function to parse the string and convert it to proper JSON
def convert_to_json(data):
    result = {}
    for key, value in data.items():
        result[key] = json.loads(value)
    return result

# Convert the data
converted_data = convert_to_json(post_data)

# Save the converted data to a JSON file
with open('associations.json', 'w') as f:
    json.dump(converted_data, f, indent=2)

print("JSON file 'associations.json' has been created successfully.")

# Optionally, print the converted data
print("\nConverted JSON data:")
print(json.dumps(converted_data, indent=2))

JSON file 'associations.json' has been created successfully.

Converted JSON data:
{
  "validAssociations": [
    {
      "companyId": "6145",
      "contactId": "6116",
      "role": "EMPLOYEE"
    },
    {
      "companyId": "25269",
      "contactId": "25232",
      "role": "EMPLOYEE"
    },
    {
      "companyId": "19716",
      "contactId": "19765",
      "role": "FOUNDER"
    },
    {
      "companyId": "8479",
      "contactId": "8485",
      "role": "ALUMNI"
    },
    {
      "companyId": "21026",
      "contactId": "21084",
      "role": "ADVISOR"
    },
    {
      "companyId": "19202",
      "contactId": "19267",
      "role": "ADVISOR"
    },
    {
      "companyId": "12333",
      "contactId": "12354",
      "role": "EMPLOYEE"
    },
    {
      "companyId": "8332",
      "contactId": "8308",
      "role": "INVESTOR"
    },
    {
      "companyId": "18679",
      "contactId": "18685",
      "role": "FOUNDER"
    },
    {
      "companyId": "2723",
      "contactId": "270

In [255]:
# Load the second JSON file
with open('associations.json', 'r') as f:
    associations_data = json.load(f)

In [256]:
# Step 4: Send an HTTP POST request
post_url = 'https://candidate.hubteam.com/candidateTest/v3/problem/result?userKey=d173419e5afe26691b3a8a08157a'
response = requests.post(post_url, json=associations_data)


print(response.status_code)

400
